In [1]:
import os 
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from keras import models
from keras import layers
from keras import optimizers
from keras.utils import normalize
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [2]:
# from Prateeks NN notebook

# set the variables for the parent data folder and train file
dataFolder = '/home/BBDC-GTA/bbdc_2019_Bewegungsdaten/'
trainData = 'train.csv'

# make a list of all the csv files in the parent and sub directories
result = list(Path(dataFolder).rglob("Subject*.[cC][sS][vV]"))

In [141]:
# Adaptation of Prateeks code for generating dataset

# Number of files to load
numFiles = 6385

# Maximum file size
maxlen = 8000

# NP Arrays to store the data and the label
labelArr = np.empty((numFiles, 1))
npArr = np.empty((numFiles, 160, 19))
indxArr = []

# read the training data into a dataframe
trainDf = pd.read_csv(dataFolder+trainData)
trainDf = trainDf[trainDf.Label != 'lay']
# factorize the labels
factor = pd.factorize(trainDf.Label)
trainDf.Label = factor[0]


cnt = 0
for eachFile in result:
  # Only read the dataset with labels available
    if sum(str(eachFile)[-35:] == trainDf.Datafile)>0:
        indxArr.append(str(eachFile)[-35:])
        file = np.loadtxt(str(eachFile), delimiter=',')
        #min max scale the inpuit
        file = (file - file.mean(axis=1, keepdims=True))
        
        #zero pad to conserve last values
        #padFile = np.zeros((50, 19))
        #file = np.concatenate((file, padFile))
        
        #windowing
        df = pd.DataFrame(file)
        emg = df.iloc[:, :4]
        other = df.iloc[::10, 4:]
        emg = emg ** 2
        emg = emg.rolling(100).mean().dropna(axis = 0, how = 'any').reset_index(drop = True)
        emg = emg.iloc[::50, :].reset_index(drop = True)
        emg = emg ** 2
        other = other.rolling(10).mean().dropna(axis = 0, how ='any').iloc[::5, :].reset_index(drop = True)
        df = pd.concat([emg, other], axis = 'columns')
        
        file = df.values
        #extend to max length
        file = (file - file.min(axis=1, keepdims=True)) / (file.max(axis = 1, keepdims = True) - file.min(axis = 1, keepdims = True))
        padFile = np.zeros((159, 19))
        file = np.concatenate((file, padFile))
        file = file[:160]
        
        data = np.expand_dims(file, axis=0)
        npArr[cnt] = data
        labelArr[cnt] = trainDf[trainDf.Datafile == str(eachFile)[-35:]].Label
        cnt += 1

In [57]:
#create test array

# Maximum file size
maxlen = 8000

# read the training data into a dataframe
testDf = pd.read_csv(dataFolder+'challenge.csv')

#number of files to read
numFiles = testDf.shape[0]

testArr = np.empty((numFiles, 160, 19))
indyArr = []

In [58]:
cnt = 0
for eachFile in result:
  # Only read the dataset with labels available
    if sum(str(eachFile)[-35:] == testDf.Datafile)>0:
        indyArr.append(str(eachFile)[-35:])
        file = np.loadtxt(str(eachFile), delimiter=',')
        #min max scale the inpuit
        file = (file - file.mean(axis=1, keepdims=True))
        
        #zero pad to conserve last values
        padFile = np.zeros((50, 19))
        file = np.concatenate((file, padFile))
        
        #windowing
        df = pd.DataFrame(file)
        emg = df.iloc[:, :4]
        other = df.iloc[::10, 4:]
        emg = emg ** 2
        emg = emg.rolling(100).mean().dropna(axis = 'index').iloc[::50, :].reset_index(drop = True)
        other = other.rolling(10).mean().dropna(axis = 'index').iloc[::5, :].reset_index(drop = True)
        df = pd.concat([emg, other], axis = 1)
        
        file = df.values
        #extend to max length
        file = (file - file.min(axis=1, keepdims=True)) / (file.max(axis = 1, keepdims = True) - file.min(axis = 1, keepdims = True))
        padFile = np.zeros((159, 19))
        file = np.concatenate((file, padFile))
        file = file[:160]
        
        data = np.expand_dims(file, axis=0)
        testArr[cnt] = data
        cnt += 1

In [59]:
print(npArr.shape)
#print(labelArr.shape)
print(len(indxArr))
print(testArr.shape)
print(len(indyArr))

(6385, 160, 19)
6385
(1738, 160, 19)
1738


In [60]:
# one-hot encode label
from keras.utils import to_categorical
enc_labelArr = to_categorical(labelArr)
enc_labelArr.shape

(6385, 22)

In [61]:
#saving and loading
np.save('npArr', npArr)
np.save('enc_labelArr', enc_labelArr)
np.save('indxArr', indxArr)
np.save('testArr', testArr)
np.save('indyArr', indyArr)



#npArr = np.load('npArr.npy')
#enc_labelArr = np.load('enc_labelArr.npy')
#indxArr = np.load('indxArr.npy')
#testArr = np.load('testArr.npy')
#indyArr = np.load('indyArr.npy')

In [142]:
print(np.isnan(np.sum(npArr)))
npArr

True


array([[[1.47345597e-01, 1.44226724e-01, 1.43087830e-01, ...,
         0.00000000e+00, 1.89424208e-09, 1.70055990e-09],
        [1.60306486e-01, 1.77104264e-01, 1.61159440e-01, ...,
         0.00000000e+00, 1.17674168e-09, 5.64717544e-10],
        [1.35275586e-01, 1.50035594e-01, 1.49557060e-01, ...,
         0.00000000e+00, 6.81040809e-10, 7.32118870e-10],
        ...,
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[6.48820909e-02, 1.18077136e-01, 8.20440598e-02, ...,
         1.08087166e-09, 6.13634854e-10, 3.27523261e-09],
        [4.69869015e-02, 1.12935200e-01, 5.35557489e-02, ...,
         6.09639444e-10, 4.68691372e-10, 1.45073843e-09],
        [1.03961372e-01, 

In [138]:
#train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(npArr, enc_labelArr, test_size = 0.2, random_state=7)
print(X_train.shape)
print(y_train.shape)

(5108, 160, 19)
(5108, 22)


### 1D Conv Model

In [63]:
#specify hyperparameters
num_classes = 22

In [66]:
# create CNN structure
model = models.Sequential()
model.add(layers.Conv1D(32, kernel_size=10, activation='relu', input_shape=(npArr.shape[1],npArr.shape[2])))
model.add(layers.SpatialDropout1D(rate=0.2))
#model.add(layers.BatchNormalization())

model.add(layers.Conv1D(64, kernel_size=5, activation='relu'))
model.add(layers.SpatialDropout1D(rate=0.4))
#model.add(layers.BatchNormalization())

#model.add(layers.Conv1D(256, kernel_size=4, activation='relu'))
#model.add(layers.SpatialDropout1D(rate=0.4))
#model.add(layers.BatchNormalization())

model.add(layers.Conv1D(128, kernel_size=4, activation='relu'))
model.add(layers.SpatialDropout1D(rate=0.4))
#model.add(layers.BatchNormalization())

#model.add(layers.Conv1D(64, kernel_size=4, activation='relu'))
#model.add(layers.SpatialDropout1D(rate=0.4))
#model.add(layers.BatchNormalization())

model.add(layers.Flatten())
model.add(layers.Dense(5000, activation = 'relu'))
model.add(layers.Dropout(rate=0.5))
model.add(layers.Dense(num_classes, activation= 'softmax'))
callbacks = [EarlyStopping(monitor="val_loss", patience=10), ModelCheckpoint(filepath="best_cnn_model.h5", 
                                                                           monitor="val_loss",save_best_only=True)]

adam = optimizers.Adam(lr = 0.001, decay = 0.0001)

#compile model using accuracy to measure model performance
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [67]:
#specify hyperparameters
epochs = 200
batch_size = 32

history = model.fit(npArr, enc_labelArr, epochs=epochs, verbose=1, batch_size=batch_size, 
                      callbacks=callbacks, validation_split = 0.2)

Train on 5108 samples, validate on 1277 samples
Epoch 1/200
 416/5108 [=>............................] - ETA: 3:45 - loss: nan - acc: 0.0409

KeyboardInterrupt: 

### Maybe 2D Conv Model?
maybe 1D Conv Layer then a couple 2D Conv?

### Regular MLP

In [13]:
#load dataset
feature_set = pd.read_csv(dataFolder + 'new_feature_df.csv')
feature_set.replace(np.NaN, 0.0, inplace=True)
feature_set.replace(np.inf, 4.0, inplace=True)

# get trainDf again if top isn't run fully
trainDf = pd.read_csv(dataFolder+trainData)
trainDf = trainDf[trainDf.Label != 'lay']

#merge labels to get training set
feature_set = pd.merge(feature_set, trainDf, left_on='id', right_on = 'Datafile', how = 'inner')
fileorder = feature_set['id']

# one-hot encode label
from keras.utils import to_categorical
factor = pd.factorize(feature_set.Label)
feature_set.Label = factor[0]
labels = to_categorical(feature_set['Label'].values)

feature_set.drop(columns= ['id', 'Subject', 'Datafile'], inplace = True)


#normalize feature set
f = feature_set.values.astype('float32')
f = normalize(f, axis = 1)
f[:5]

array([[ 1.6515067e-02, -6.7485064e-02, -4.0974207e-02, ...,
         3.4078300e-07, -2.1590799e-06,  0.0000000e+00],
       [ 2.1176578e-02, -2.7541181e-02, -4.2534634e-02, ...,
        -6.9564326e-06,  4.9421424e-06,  9.0536887e-06],
       [ 5.3427730e-02, -1.5425559e-02, -2.5265059e-02, ...,
        -1.6132236e-06,  7.0478754e-06,  2.3184088e-05],
       [ 5.1594611e-02, -3.2933217e-02, -8.1078112e-02, ...,
        -6.1627206e-06, -1.0733088e-06,  7.5541160e-05],
       [ 2.4223570e-02, -3.6726356e-02, -3.2027334e-02, ...,
         1.7665525e-06,  7.1124373e-07,  3.2856657e-05]], dtype=float32)

In [58]:
# create MLP structure
mlp = models.Sequential()
mlp.add(layers.Dense(2640, activation='relu', input_dim=264))
mlp.add(layers.Dropout(rate=0.2))
mlp.add(layers.Dense(1280, activation = 'relu'))
mlp.add(layers.Dropout(rate=0.5))
mlp.add(layers.Dense(800, activation = 'relu'))
mlp.add(layers.Dropout(rate=0.5))
mlp.add(layers.Dense(400, activation = 'relu'))
mlp.add(layers.Dropout(rate=0.5))
mlp.add(layers.Dense(22, activation= 'softmax'))

callbacks = [EarlyStopping(monitor="val_loss", patience=10), ModelCheckpoint(filepath="best_mlp_model.h5", 
                                                                           monitor="val_loss",save_best_only=True)]

adam = optimizers.Adam(lr = 0.001)
sgd = optimizers.SGD(lr = 0.001, momentum = 0.1, decay = 0.001)
#compile model using accuracy to measure model performance
mlp.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [59]:
epochs = 200

batch_size = 32

hist = mlp.fit(f, labels, batch_size=batch_size, epochs=epochs, callbacks=callbacks, 
          validation_split=0.2)

Train on 5108 samples, validate on 1277 samples
Epoch 1/200
5108/5108 [==============================] - 14s 3ms/step - loss: 2.5054 - acc: 0.1754 - val_loss: 2.0944 - val_acc: 0.2553
Epoch 2/200
5108/5108 [==============================] - 11s 2ms/step - loss: 1.6522 - acc: 0.3604 - val_loss: 1.8482 - val_acc: 0.3547
Epoch 3/200
5108/5108 [==============================] - 11s 2ms/step - loss: 1.2091 - acc: 0.4955 - val_loss: 1.2940 - val_acc: 0.4894
Epoch 4/200
5108/5108 [==============================] - 12s 2ms/step - loss: 0.9715 - acc: 0.5781 - val_loss: 1.3999 - val_acc: 0.5129
Epoch 5/200
5108/5108 [==============================] - 12s 2ms/step - loss: 0.8481 - acc: 0.6341 - val_loss: 1.1263 - val_acc: 0.5873
Epoch 6/200
5108/5108 [==============================] - 11s 2ms/step - loss: 0.8224 - acc: 0.6443 - val_loss: 1.4963 - val_acc: 0.5411
Epoch 7/200
5108/5108 [==============================] - 12s 2ms/step - loss: 0.6800 - acc: 0.7034 - val_loss: 1.2380 - val_acc: 0.5442


### CNN plus feature MLP
inspired by https://www.pyimagesearch.com/2019/02/04/keras-multiple-inputs-and-mixed-data/

In [32]:
#load dataset
feature_set = pd.read_csv(dataFolder + 'new_feature_df.csv')
feature_set.replace(np.NaN, 0.0, inplace=True)
feature_set.replace(np.inf, 4.0, inplace=True)

# get trainDf again if top isn't run fully
trainDf = pd.read_csv(dataFolder+trainData)
trainDf = trainDf[trainDf.Label != 'lay']

#merge labels to get training set
feature_set = pd.merge(feature_set, trainDf, left_on='id', right_on = 'Datafile', how = 'inner')
fileorder = feature_set['id']

feature_set.drop(columns= ['id', 'Subject', 'Datafile', 'Label'], inplace = True)

#normalize feature set
f = feature_set.values
f = (f - f.min(axis=1, keepdims=True)) / (f.max(axis=1, keepdims=True) - f.min(axis=1, keepdims=True))

In [5]:
### Network Structure
# import the necessary packages
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers.core import SpatialDropout1D
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Concatenate
from keras.layers import concatenate

#input_ts = Input(shape=(npArr.shape[1], npArr.shape[2]))
#input_f = Input(shape=f_train.shape[1],)

In [6]:
## creating CNN component
# initialize the input shape and channel dimension, assuming
# TensorFlow/channels-last ordering
    
inputShape = (npArr.shape[1], npArr.shape[2],)
chanDim = -1
 
# define the model input
input_ts = Input(shape=inputShape)
 
# CONV => RELU => BN => POOL
c = Conv1D(32, 10, strides=10, activation = 'relu')(input_ts)
c = BatchNormalization(axis=chanDim, momentum = 0.0)(c)
c = MaxPooling1D(pool_size=2)(c)
c = SpatialDropout1D(rate=0.2)(c)
    
c = Conv1D(64, 10, activation = 'relu')(c)
c = BatchNormalization(axis=chanDim, momentum = 0.0)(c)
c = MaxPooling1D(pool_size=2)(c)
c = SpatialDropout1D(rate=0.2)(c)
    
c = Conv1D(128, 4, activation = 'relu')(c)
c = BatchNormalization(axis=chanDim, momentum = 0.0)(c)
c = MaxPooling1D(pool_size=2)(c)
c = SpatialDropout1D(rate=0.2)(c)
        
# flatten the volume, then FC => RELU => BN => DROPOUT
c = Flatten()(c)
c = Dense(2000, activation='relu')(c)
c = Dropout(0.4)(c)
 
# apply another FC layer, this one to match the number of nodes
# coming out of the MLP
c_out = Dense(500, activation='relu')(c)

### create simple MLP
#model input
input_m = Input(shape=(264,))

#two dense layers with dropout in between
m = Dense(1000, activation="relu")(input_m)
m = BatchNormalization(axis=chanDim, momentum = 0.0)(m)
m = Dropout(rate=0.2)(m)
m = Dense(1000, activation="relu")(input_m)
m = Dropout(rate=0.4)(m)
m_out = Dense(500, activation="relu")(m)

# create the input to our final set of layers as the *output* of both
# the MLP and CNN
comb = concatenate([c_out, m_out])

# our final FC layer head will have two dense layers, the final one
# being our regression head
comb = Dense(1000, activation="relu")(comb)
comb = Dropout(rate = 0.5)(comb)
out = Dense(22, activation="softmax")(comb)

model = Model(inputs=[input_ts, input_m], outputs=out)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [7]:
#specify optimizers
adam = Adam(lr = 0.001, decay = 0.001 / 200)

#compile model using accuracy to measure model performance
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
#specify hyperparameters
epochs = 200
batch_size = 64

#early stopping
callbacks = [EarlyStopping(monitor="val_loss", patience=30), ModelCheckpoint(filepath="best_comb_model.h5", 
                                                                           monitor="val_loss",save_best_only=True)]

model.fit([npArr, f], enc_labelArr, batch_size=batch_size, epochs=epochs, callbacks=callbacks, 
          validation_split=0.2, shuffle = True)

Instructions for updating:
Use tf.cast instead.
Train on 5108 samples, validate on 1277 samples
Epoch 1/200
5108/5108 [==============================] - 65s 13ms/step - loss: 2.5863 - acc: 0.2101 - val_loss: 1.3896 - val_acc: 0.4839
Epoch 2/200
5108/5108 [==============================] - 63s 12ms/step - loss: 1.3617 - acc: 0.5305 - val_loss: 0.9725 - val_acc: 0.6711
Epoch 3/200
5108/5108 [==============================] - 63s 12ms/step - loss: 0.8792 - acc: 0.6962 - val_loss: 0.7243 - val_acc: 0.7439
Epoch 4/200
5108/5108 [==============================] - 63s 12ms/step - loss: 0.6472 - acc: 0.7892 - val_loss: 0.6493 - val_acc: 0.7886
Epoch 5/200
5108/5108 [==============================] - 63s 12ms/step - loss: 0.5158 - acc: 0.8289 - val_loss: 0.8706 - val_acc: 0.7619
Epoch 6/200
5108/5108 [==============================] - 63s 12ms/step - loss: 0.4650 - acc: 0.8496 - val_loss: 0.8612 - val_acc: 0.7776
Epoch 7/200
5108/5108 [==============================] - 63s 12ms/step - loss: 0.4

KeyboardInterrupt: 

### CNN plus earlier averaged RF models
Source: https://www.mdpi.com/1424-8220/16/1/115

In [2]:
from keras.models import load_model
cnn = load_model('best_cnn_model.h5')

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [23]:
cnnpred = cnn.predict(testArr)

In [69]:
print(np.argmax(cnnpred, axis = 1)[:50])

[ 8  9  4 18  4 16  3 21 16  1  3  0 16  9 13 16 11 15 14 13 15  3 19 12
 21 20 11  2 13  2  0  8 17 19 18 18 15  6  2 17 13 18  9  5 17 21 20 21
  1 15]


In [64]:
#load dataset
feature_set = pd.read_csv(dataFolder + 'new_feature_df.csv')
feature_set.replace(np.NaN, 0.0, inplace=True)
feature_set.replace(np.inf, 4.0, inplace=True)

# get trainDf again if top isn't run fully
trainDf = pd.read_csv(dataFolder+trainData)
trainDf = trainDf[trainDf.Label != 'lay']

#merge labels to get training set
train_df = pd.merge(feature_set, trainDf, left_on='id', right_on = 'Datafile', how = 'inner')
train_df.drop(columns= ['id', 'Subject', 'Datafile'], inplace = True)

factor = pd.factorize(train_df['Label'])
train_df.Label = factor[0]

#initialize dataframe with test observations
test_subjects = pd.read_csv(dataFolder + 'challenge.csv')

#create test_df by merging with testFile
test_df = pd.merge(feature_set, test_subjects, left_on='id', right_on= 'Datafile', how='inner')
test_df.drop(columns = ['Label', 'id', 'Subject', 'Datafile'], inplace = True)

print(test_df.shape)
print(train_df.shape)

(1738, 264)
(6385, 265)


In [65]:
# generate old RF models
from sklearn.ensemble import RandomForestClassifier

#train test split
f_train, f_test, l_train, l_test= train_test_split(train_df.iloc[:, -1:], train_df.iloc[:, -1], test_size = 0.2, random_state=7)

# hyperparameters
random_state_range = range(40, 60)
n_trees = 100

#initialize probability array
prob = np.empty((len(test_df), 22))

#loop to run Random Forest in different random states and add the probabilities obtained from each to prob
cnt = 0
for random_state in random_state_range:
    classifier = RandomForestClassifier(n_estimators = n_trees, random_state = random_state)
    classifier.fit(train_df.iloc[:, :-1], train_df.iloc[:, -1])
    prob += classifier.predict_proba(test_df)
    cnt += 1

#argmax function to choose the label with highest cumulative probability
cum_pred = np.argmax(prob, axis = 1)

In [68]:
cum_pred

array([17, 12,  9, ...,  1,  8,  5])

In [63]:
sum(indyArr == test_df['Datafile'])

1738

In [140]:
x = pd.DataFrame(np.zeros(300))
x = x.rolling(10).mean().dropna(axis='index').iloc[::10].apply(np.sqrt).reset_index(drop = True)

y = pd.DataFrame(list(range(600)))
y = y.rolling(10).mean().dropna(axis='index').iloc[::20].apply(np.sqrt).reset_index(drop = True)

xy = pd.concat([x, y], axis = 1)
xy

,0,0
0,0.0,2.121320
1,0.0,4.949747
2,0.0,6.670832
3,0.0,8.031189
4,0.0,9.192388
5,0.0,10.222524
6,0.0,11.157957
7,0.0,12.020815
8,0.0,12.825755
9,0.0,13.583078


In [77]:
f = xy.values
f

array([[ 2.12132034,  2.12132034],
       [ 3.80788655,  4.94974747],
       [ 4.94974747,  6.67083203],
       [ 5.87367006,  8.0311892 ],
       [ 6.67083203,  9.19238816],
       [ 7.38241153, 10.22252415],
       [ 8.0311892 , 11.1579568 ],
       [ 8.63133825, 12.02081528],
       [ 9.19238816, 12.82575534],
       [ 9.72111105, 13.58307771],
       [10.22252415, 14.30034965],
       [10.70046728, 14.98332406],
       [11.1579568 , 15.63649577],
       [11.5974135 , 16.26345597],
       [12.02081528, 16.86712779],
       [12.42980289, 17.44992837],
       [12.82575534, 18.01388353],
       [13.20984481, 18.56071119],
       [13.58307771, 19.09188309],
       [13.94632568, 19.60867155],
       [14.30034965, 20.11218536],
       [14.64581852, 20.60339778],
       [14.98332406, 21.08316864],
       [15.31339283, 21.55226206],
       [15.63649577, 22.0113607 ],
       [15.95305613, 22.46107745],
       [16.26345597, 22.90196498],
       [16.56804153, 23.33452378],
       [16.86712779,

In [122]:
arr = array - array.mean(axis = 1, keepdims = True)
arr

array([[-2., -1.,  0.,  1.,  2.],
       [-2., -1.,  0.,  1.,  2.],
       [-2., -1.,  0.,  1.,  2.],
       [-2., -1.,  0.,  1.,  2.]])

In [135]:
arr = pd.DataFrame(arr)
arr = arr.rolling(2, axis = 0).mean().dropna(axis = 0).iloc[::5, :].reset_index(drop = True)

arr

,0,1,2,3,4


In [131]:
arr.iloc[::3, :].reset_index(drop = True)

,0,1,2,3,4
0,4.0,1.0,0.0,1.0,4.0


In [130]:
arr = arr ** 2
arr.apply(np.sqrt)

,0,1,2,3,4
1,2.0,1.0,0.0,1.0,2.0
2,2.0,1.0,0.0,1.0,2.0
3,2.0,1.0,0.0,1.0,2.0
